# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [26]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/yurtemre/Library/Python/3.11/lib/python/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['permutation', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [27]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Right', 'Crouch', 'Back', 'StandInit', 'Sit', 'Left', 'Belly', 'Stand', 'Knee', 'HeadBack', 'Frog']


In [28]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, "rb"))
    target = [i] * len(data)
    return data, target

In [29]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data += data
    all_target += target


print('total number of data', len(all_data))

total number of data 222


In [30]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [31]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [34]:
# YOUR CODE HERE
x = [all_data[i] for i in training_data]
y = [all_target[i] for i in training_data]
clf.fit(x, y)

SVC(C=100.0, gamma=0.001)

### predicting

In [38]:
clf.predict([all_data[-1]]), all_target[-1]

(array([10]), 10)

In [39]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [40]:
expected = []
predicted = []
# YOUR CODE HERE
for i in range(len(all_data)):
    if i not in training_data:
        expected.append(all_target[i])
        predicted.append(clf.predict([all_data[i]])[0])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.89      1.00      0.94         8
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         5
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         4

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.98        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 3  1  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [41]:
expected = []
predicted = []
# YOUR CODE HERE
for i in range(len(all_data)):
    if i not in training_data:
        expected.append(all_target[i])
        predicted.append(clf.predict([all_data[i]])[0])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.89      1.00      0.94         8
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00         5
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00         4

    accuracy                           0.99        67
   macro avg       0.99      0.98      0.98        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 3  1  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [ ]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

: 

Then, in the application we can load the trained classifier again.

In [ ]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

: 

: 